In [12]:
import numpy as np
import pandas as pd
from dataloaders import *
from utils import *
from EDA import *
import config

In [2]:
dataset_dir = config.dataset_dir
subject_id = np.arange(10)+1
sic_candidate = np.arange(30)+1

In [ ]:
bold, _ = data_import(dataset    = dataset_dir,
                      subject_id = subject_id,
                      subfolder  = 'Analysis.ica',
                      data_name  = ['filtered_func_data'])

fmri_data = bold['filtered_func_data'].get_fdata()

sica, _ = data_import(dataset    = dataset_dir,
                             subject_id = subject_id,
                             subfolder  = 'Analysis.ica\\filtered_func_data.ica',
                             data_name  = ['mask','melodic_IC'])

selected_ic = ic_extract(sica['melodic_IC'],sic_candidate)

_, selected_ic_cluster = ic_browse(selected_ic = selected_ic,
                                   criterion   = selected_ic >= 3.1,
                                   affine      = sica['melodic_IC'].affine,
                                   show        = False)

selected_ic_cluster = morphological_operation(selected_ic_cluster)

max_value, _ = ic_browse(selected_ic = selected_ic,
                         criterion   = selected_ic >= selected_ic.max(),
                         affine      = sica['melodic_IC'].affine,
                         show        = False)

LI_result = lateralization_index(selected_ic)
BLI_result = binary_lateralization(selected_ic_cluster)
selected_ic_mask = mask_cc_around_coordinate(selected_ic_cluster, max_value["Max_index"][0])
connectivity_matrix = cluster_connectivity_ds(fmri_data, selected_ic_mask, downsample_factor=5)
central_network_strength = voxel_network_strength(fmri_data, selected_ic_mask, max_value["Max_index"][0])
random_matrices = generate_random_matrices(connectivity_matrix, num_random_matrices=500)
TLI_result = compute_topological_measures(connectivity_matrix, random_matrices)

In [10]:
sIC_Features = {
    'Subject ID': subject_id,
    'sIC Number': sic_candidate,
    'sIC Center': max_value['MNI_coords'],
    'Lateralization Index': LI_result['Lateralization Index'],
    'Lateralization Strength': BLI_result['Lateralization Strength'],
    'Lateralization Side': BLI_result['Side'],
    'Centeral Network': central_network_strength,
    'Clustering Coefficient': TLI_result['Clustering Coefficient'],
    'Characteristic Path Length': TLI_result['Characteristic Path Length'],
    'Local Efficiency': TLI_result['Local Efficiency'],
    'Connectivity Strength': TLI_result['Connectivity Strength'],
    'Connectivity Diversity': TLI_result['Connectivity Diversity'],  
}

df = pd.DataFrame(sIC_Features, index=[0])
results_path = f'results/Sub{subject_id}_sIC_Features.csv'

if os.path.isfile(results_path):
    df.to_csv(results_path, mode='a', index=False, header=False)
else:
    df.to_csv(results_path, index=False, header=True)